In [ ]:
import sys
import os
from code import InteractiveConsole

class SyntacticAnalyzer:
    def __init__(self):
        self.console = InteractiveConsole()
        self.instructions = []
        self.indentation_level = 0  # Track indentation for multi-line statements

    def is_complete(self, line):
        # Check if the line ends with a colon (for loops, conditionals)
        if line.strip().endswith(':'):
            self.indentation_level += 1
            return False
        elif self.indentation_level > 0:
            # Check for indentation level decrease (indicating end of block)
            if not line.startswith(' ' * 4):  # Assuming 4 spaces for indentation
                self.indentation_level -= 1
                return True
            return False
        return True

    def execute_line(self, line):
        try:
            if self.console.push(line):
                # If it's an incomplete statement, buffer it
                self.instructions.append(line)
                print("\t", end='')  # Indicate waiting for more input
            else:
                # Complete statement; execute buffered instructions
                self.instructions.append(line)
                full_code = ''.join(self.instructions)
                exec(full_code)  # Execute the full code block
                self.instructions.clear()  # Clear buffer after execution
                print("Executed successfully.")
        except Exception as e:
            print(f'Error: {e}')
            self.instructions.clear()  # Clear buffer on error

def main():
    analyzer = SyntacticAnalyzer()
    print('Toy test language interpreter')

    while True:
        try:
            line = sys.stdin.readline()
            if len(line) == 0:  # End of input
                sys.exit(0)

            analyzer.execute_line(line)

        except KeyboardInterrupt:
            print("\nExiting...")
            break

if __name__ == "__main__":
    main()


Toy test language interpreter
